In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction
In this notebook, we will show you how to build a Dialogflow CX Agent from scratch using plain text inputs and a simple CSV file.

## Prerequisites
- Ensure you have a GCP Service Account key with the Dialogflow API Admin privileges assigned to it.

In [ ]:
#If you haven't already, make sure you install the `dfcx-scrapi` library

!pip install dfcx-scrapi

# Imports

In [ ]:
import pandas as pd

from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.core.flows import Flows
from dfcx_scrapi.core.pages import Pages
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions
from dfcx_scrapi.tools.maker_util import MakerUtil

creds_path = '<YOUR_CREDS_PATH_HERE>'

# Method 1 - Creating an Agent from Simple Text Inputs

## Create Your Agent
Creating an agent requires a minimum of 2 pieces of information:
- `project_id`, which is your GCP Project ID
- `display_name`, (i.e. 'My Cool Agent!')
- `gcp_region`, (Optional) This defaults to `global` region, but you can provide any GCP region that is currently available for Dialogflow CX.

In [ ]:
# First we will instansiate our Agent object
a = Agents(creds_path=creds_path)

# Next, we will set some variables for our agent creation args
project_id = '<YOUR_GCP_PROJECT_ID>'
display_name = 'My Cool Agent!'
gcp_region = 'us-central1'

# Then we will call the `create_agent` method and capture the result in a var call `my_agent`
my_agent = a.create_agent(project_id, display_name, gcp_region)


If you inspect your `my_agent` variable, you will see it is of types.agent.Agent
and contains a structure similar to this below

```
name: "projects/<your-gcp-project>/locations/us-central1/agents/08e7647e-8b6e-4f1e-abdb-1d0448127a05"
display_name: "My Cool Agent!"
default_language_code: "en"
time_zone: "America/Chicago"
start_flow: "projects/<your-gcp-project>/locations/us-central1/agents/08e7647e-8b6e-4f1e-abdb-1d0448127a05/flows/00000000-0000-0000-0000-000000000000"
```

## Create Your First Intent
For this demo agent, we'll build a basic intent from list of Training Phrases (TPs)

To simplify the Intent creation, we'll utilize the `DataframeFunctions` class from the `tools` portion of the SCRAPI library.   
This will allow us to build our intent into a simple Pandas DataFrame, and then push this DataFrame directly into our bot that we just created.

A common method of building Intents and Training Phrases for Dialogflow CX agents is to use Google Sheets or CSVs to store the Intent/TP data.  
For this demo, we've included a file located at `data/sample_intent_tp.csv` that we'll pull into our dataframe to use.

In [ ]:
# First, we will instantiate our DataframeFunctions (dffx) object
dffx = DataframeFunctions(creds_path=creds_path)

# Next, we will read in our sample CSV with Intent/TP data into a Pandas DataFrame
df = pd.read_csv('../data/sample_intent_tp.csv')

# Finally, we will use `dffx` to push our Intents to our Agent
my_intents = dffx.bulk_create_intent_from_dataframe(my_agent.name, df, update_flag=True)

## Create Your First Page
Next, we'll use the `create_page` method to create a Page in our Default Start Flow and attach our newly created Intent to it.

For Page object manipulation, you'll want to know your Flow ID metadata.   
We'll use the `Flows` class and `get_flows_map` method to provide this info.

In [ ]:
# First, we will instantiate our Flows and Pages objects.
f = Flows(creds_path=creds_path)
p = Pages(creds_path=creds_path)

# The `get_flows_map` method provides an easy to use map of your Flows, their IDs, and their human readable Display Names
# Using the `reverse=True` arg allows you to call the dictionary by your flow Display Names which can be easier for exploratory building.
flows_map = f.get_flows_map(my_agent.name, reverse=True)

# Now that we know our Flow ID, we'll create a Page using the Flow ID as the Page parent
my_page = p.create_page(flows_map['Default Start Flow'], display_name='My First Page!')


If you inspect the `my_page` object, you will see it is of types.page.Page and contains a structure similar to this below

```
name: "projects/<your-gcp-project>/locations/us-central1/agents/fc5e72f6-c194-43ec-871f-6a636010f4ae/flows/00000000-0000-0000-0000-000000000000/pages/63037247-d3f2-42e5-8c43-651bcbcf2c0e"
display_name: "My First Page!"
```

## Add Intent Route to Default Start Flow and Link Page
In our last step, we will link all of our previous steps together by:
- Adding our `head_intent.order_pizza` intent to our `Start` Page of our `Default Start Flow` as a `Transition Route`
- Creating a `Target Page` pointing to our `My First Page!` within our `Transition Route` 

The `Start` page of every `Flow` is a special type of page and is actually a part of the `Flow` resource object itself.  
So in order to add the `Transition Route`, we will be manipulating the `Default Start Flow` flow object directly.

To help us, we'll also use the `maker_util` from the `tools` directory to build our `Transition Route` object.  
We'll need 2 main pieces of information for our `Transition Route`:
- Our Intent ID
- Our Page ID

Dialogflow Resource IDs can always be found in the `x.name` namespace.  
For our intent, it would be `my_intent.name`  
For our page, it would be `my_page.name`   

In [ ]:
# We'll first get the Default Start Flow (dsf for short)
dsf = f.get_flow(flows_map['Default Start Flow'])

# We'll also fetch our Intent directly from the bot by Display Name to make this next part easier
i = Intents(creds_path=creds_path)
intent_list = i.list_intents(my_agent.name)
my_intent = [x for x in intent_list if x.display_name == 'head_intent.order_pizza'][0]

# Next, we'll use the MakerUtil to build our Transition Route (tr) object
mu = MakerUtil()
my_tr = mu.make_transition_route(intent=my_intent.name, target_page=my_page.name)

# Now, we'll add the newly created Transition Route object to our DSF object
dsf.transition_routes.append(my_tr)

# Finally, we'll update the DSF object in our agent
my_updated_flow = f.update_flow(flows_map['Default Start Flow'], obj=dsf, transition_routes=dsf.transition_routes)

# Bot Building 101 End
## And there you have it!   
We've created a simple Dialogflow CX agent using only Python in a Jupyter notebook.  
You can see how this could be easily scaled up using .py files, git repos, and other scripts to speed up the bot building process.

If you're ready to dive into a more advanced style of bot building, check out the Bot Building 201 notebook which shows you how to build a Dialogflow CX agent using a YAML file.